## Version 1: Weight it based on 1X2 probabilities

In [1]:
def calculate_expected_goals(prob_a_winning, prob_b_winning, prob_draw, total_goals):
    # Calculate the expected goals for each outcome
    goals_team_a_winning = total_goals * prob_a_winning
    goals_team_b_winning = total_goals * prob_b_winning
    goals_draw = total_goals * prob_draw

    # Sum the expected goals for each team
    expected_goals_team_a = goals_team_a_winning + (0.5 * goals_draw)
    expected_goals_team_b = goals_team_b_winning + (0.5 * goals_draw)

    return expected_goals_team_a, expected_goals_team_b

In [2]:
# Example probabilities and total goals
prob_a_winning = 0.17543859649122806
prob_b_winning = 0.5988023952095809
prob_draw = 0.22727272727272727
total_goals = 2.633995

expected_goals_team_a, expected_goals_team_b = calculate_expected_goals(prob_a_winning, prob_b_winning, prob_draw, total_goals)

print("Expected goals for Team A:", expected_goals_team_a)
print("Expected goals for Team B:", expected_goals_team_b)

Expected goals for Team A: 0.7614219996012759
Expected goals for Team B: 1.8765601286064237


## Version 2 : Find optimal average goals per team, based on target win probabilities

In [9]:
# This script finds the optimal values for 𝜇𝐴 and 𝜇𝐵 that best match the target win probabilities and then uses these values to 
# calculate the expected goals for the Home and Away teams.
# Notation: 𝜇𝐴 = mean/average goals scored by Team A

import numpy as np
from scipy.stats import poisson

def calculate_expected_goals(mu_A, mu_B, max_goals=10):
    prob_A = [poisson.pmf(k, mu_A) for k in range(max_goals + 1)]
    prob_B = [poisson.pmf(k, mu_B) for k in range(max_goals + 1)]
    
    E_A = sum(k * prob_A[k] for k in range(max_goals + 1))
    E_B = sum(k * prob_B[k] for k in range(max_goals + 1))
    
    return E_A, E_B

def optimize_expected_goals(target_p_A, target_p_B, mu_total, epsilon=0.1, max_goals=10):
    mu_A_values = np.arange(0, mu_total + epsilon, epsilon)
    best_fit = None
    best_error = float('inf')
    
    for mu_A in mu_A_values:
        mu_B = mu_total - mu_A
        P_A_wins, P_B_wins, P_draw = calculate_win_draw_probabilities(mu_A, mu_B, max_goals)
        
        error = (P_A_wins - target_p_A) ** 2 + (P_B_wins - target_p_B) ** 2
        if error < best_error:
            best_error = error
            best_fit = (mu_A, mu_B)
    
    best_mu_A, best_mu_B = best_fit
    E_A, E_B = calculate_expected_goals(best_mu_A, best_mu_B, max_goals)
    
    return best_mu_A, best_mu_B, E_A, E_B

def calculate_win_draw_probabilities(mu_A, mu_B, max_goals=10):
    prob_A = [poisson.pmf(k, mu_A) for k in range(max_goals + 1)]
    prob_B = [poisson.pmf(k, mu_B) for k in range(max_goals + 1)]
    
    P_A_wins = 0
    P_B_wins = 0
    P_draw = 0
    
    for i in range(max_goals + 1):
        for j in range(max_goals + 1):
            if i > j:
                P_A_wins += prob_A[i] * prob_B[j]
            elif i < j:
                P_B_wins += prob_A[i] * prob_B[j]
            else:
                P_draw += prob_A[i] * prob_B[j]
    
    return P_A_wins, P_B_wins, P_draw

# Example usage
target_p_A = 0.17543859649122806
target_p_B = 0.5988023952095809
mu_total = 2.633995
epsilon = 0.05

best_mu_A, best_mu_B, E_A, E_B = optimize_expected_goals(target_p_A, target_p_B, mu_total, epsilon)
print(f"Best fit mu_A: {best_mu_A}, mu_B: {best_mu_B}")
print(f"Expected goals for Home Team (A): {E_A}")
print(f"Expected goals for Away Team (B): {E_B}")

# Optional: Zoom in for a smaller epsilon around the best fit
epsilon_zoom = 0.01
mu_total_zoom = best_mu_A + best_mu_B
best_mu_A_zoom, best_mu_B_zoom, E_A_zoom, E_B_zoom = optimize_expected_goals(target_p_A, target_p_B, mu_total_zoom, epsilon_zoom)

print(f"Zoomed in best fit mu_A: {best_mu_A_zoom}, mu_B: {best_mu_B_zoom}")
print(f"Expected goals for Home Team (A): {E_A_zoom}")
print(f"Expected goals for Away Team (B): {E_B_zoom}")


Best fit mu_A: 0.8500000000000001, mu_B: 1.783995
Expected goals for Home Team (A): 0.8499999786511608
Expected goals for Away Team (B): 1.7839629104561419
Zoomed in best fit mu_A: 0.84, mu_B: 1.7939950000000002
Expected goals for Home Team (A): 0.839999981087032
Expected goals for Away Team (B): 1.7939611795736825
